# Stage 1 Funnel - Identify promising KS Portfolio combinations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import seaborn as sns
import datetime as dt
import os
import re
import os, sys, importlib, math, string
import connector as hk

## 1. Identify portfolios with a high percentage of spend on high impressionshare campaigns

__pull campaign and portfolio ids within the KS during the time frame being considered__

In [2]:


def identifier(KSlist, spend_thrshold_per, start_date, end_date):
    exceptions = []
    d = pd.DataFrame(columns=['portfolio_id','cost','ports','%spendhighimprshare','cost_on_high_imprshare'])
    exceptions = []
    for KS in KSlist:
        try:
        
            K = KS[2:]
            num = K
            num = re.sub(r'^11', "41", num)
            num = re.sub(r'^12', "42", num)
            num = re.sub(r'^15', "40", num)
            ks_id = num
            host = 'eclidb'+str(ks_id)+'.kenshooprd.local'
            _mysql_connector = __import__('mysql.connector').connector if sys.version_info.major == 2 else __import__('pymysql')
            cnx = _mysql_connector.connect(user='query',password='query', database='kazaam',host=host)
    
            portmembers = f'''
            select campaign_id, portfolio_id
            from portfolio_members 
            where start_date < '{start_date}'
            and end_date is NULL
            '''
            portmember = pd.DataFrame(pd.read_sql(portmembers, con=cnx))
            CampaignList = portmember['campaign_id']
            if len(CampaignList) == 0:
                continue
            CampaignList_SQL = ','.join(map(str, CampaignList))
        
        
    
            queryPerfCamp =f''' 
            select *
            from performance_by_campaign_id
            where campaign_id in ({CampaignList_SQL})
            and date BETWEEN '{start_date}' and '{end_date}'
            and cost > 0
               
            '''
            PerfCamp = pd.DataFrame(pd.read_sql(queryPerfCamp, con=cnx))
    
            df = pd.merge(PerfCamp, portmember, how = 'inner', left_on = ['campaign_id'], right_on = ['campaign_id'])
            dft = df.groupby(['portfolio_id','campaign_id']).agg({'imprShare':'median', 'cost':'sum', 'clicks': 'sum'})
            dft = dft.reset_index()
            portlist = df.portfolio_id.unique()
        
            hists = []
            for portfolio in portlist:
                #create a unique dataframe for each portfolio
                portdf = dft[(dft['portfolio_id'] == portfolio)]
                #within each portfolio calculate the sum of spend on high imprshare campaigns (>0.98) divided by the overall spend 
                hists.append(portdf[portdf['imprShare'] > 0.98]['cost'].sum()/ portdf['cost'].sum())
            ports = pd.DataFrame(zip(portlist, hists), columns = ['ports', '%spendhighimprshare'])
            portspend = dft.groupby('portfolio_id').agg({'cost': 'sum'})
            portspend = portspend.reset_index()
    
            costs_allports = pd.merge(portspend, ports, left_on = 'portfolio_id', right_on = 'ports', how = 'inner')
            costs_allports['cost_on_high_imprshare'] = costs_allports['cost']*costs_allports['%spendhighimprshare']
            table = costs_allports.sort_values('cost_on_high_imprshare', ascending = False)
            #filter for KS/Portfolio combinations with 'spend_threshold_per' or more of their spend on high impressionshare campaigns
            table = table[table['%spendhighimprshare'] >= spend_thrshold_per]
            table['KS'] = KS
            d = pd.concat(objs = [d,table], sort = True)
        
            d= d.sort_values(by = '%spendhighimprshare', ascending = False)
            d= d.sort_values(by = 'cost_on_high_imprshare', ascending = False)
        except:
            exceptions.append(KS)
    return d

## Get list of ksnames

In [3]:
sys.path.insert(0, '../../classes/')



importlib.reload(hk)
ksc = hk.KSC()

In [4]:
q = """select DISTINCT(ksname) from dwh_prod.performance_by_portfolio
 """ 

print(q)

select DISTINCT(ksname) from dwh_prod.performance_by_portfolio
 


In [5]:
ksdf = ksc.query('hive', q)
len(ksdf)

570

In [6]:
ksdf.head()

,ksname
0,KS1101
1,KS1102
2,KS1104
3,KS1105
4,KS1106


## rank portfolios by cost on high impressionshare

In [7]:
table = identifier(ksdf['ksname'], 0.95, '2019-10-01', '2019-10-31')
table.head()

,%spendhighimprshare,KS,cost,cost_on_high_imprshare,portfolio_id,ports
3,1.000000,KS5103,586548.959778,586548.959778,21.0,21.0
0,0.976331,KS3316,565199.531829,551821.620163,3.0,3.0
0,1.000000,KS7156,257129.555176,257129.555176,54.0,54.0
0,1.000000,KS3651,212660.249023,212660.249023,1.0,1.0
4,1.000000,KS3736,84847.439682,84847.439682,7.0,7.0


In [8]:
table.to_csv('clickshare_stage_one_funnel.csv')